# Combining Files

Note: get the data files here [https://data.gov.sg/dataset/resale-flat-prices](https://data.gov.sg/dataset/resale-flat-prices).

In [21]:
import pandas as pd
import os
import glob
import pathlib
from timeit import default_timer as timer

In [6]:
os.getcwd()

'C:\\Users\\Tobias Leong\\Code\\resale-price-analysis'

In [7]:
os.listdir('data')

['.ipynb_checkpoints',
 'resale-flat-prices-based-on-approval-date-1990-1999.csv',
 'resale-flat-prices-based-on-approval-date-2000-feb-2012.csv',
 'resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv',
 'resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv',
 'resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv']

In [8]:
glob.glob('data/*.csv')

['data\\resale-flat-prices-based-on-approval-date-1990-1999.csv',
 'data\\resale-flat-prices-based-on-approval-date-2000-feb-2012.csv',
 'data\\resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv',
 'data\\resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv',
 'data\\resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv']

In [9]:
df = pd.DataFrame()
for datafile in glob.glob('data/*.csv'):
    tmp_df = pd.read_csv(datafile)
    df = df.append(tmp_df)
print(f'Resulting dataframe of {df.shape} shape')

Resulting dataframe of (842065, 11) shape


In [28]:
# save df into file 
# overwrite if necessary
def write_file(path, override=True):
    if override:
        curr_path = pathlib.Path(path)
        if curr_path.is_file():
            print(f"Existing file found at {curr_path}, deleting...")
            os.remove(curr_path)
            print("Delete complete!")
    start = timer()
    df.to_csv(path, index=False)
    end = timer()
    print(f"File written at {curr_path} in {(end - start):.2f}s")

In [29]:
p = 'out/combined_df.csv'
write_file(p)

Existing file found at out\combined_df.csv, deleting...
Delete complete!
File written at out\combined_df.csv in 1.83s


# Analysis

In [32]:
df.head(5)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN


In [33]:
df.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'remaining_lease'],
      dtype='object')

In [34]:
df.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
resale_price                0
remaining_lease        709054
dtype: int64

Many `remaining_lease` are missing!

In [42]:
# create new column year 
def get_year_from_month_col(row):
    year = row['month'].split('-')[0]
    return year

In [43]:
df['year'] = df.apply(get_year_from_month_col, axis=1)

In [44]:
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,year
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN,1990
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN,1990
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN,1990
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN,1990
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN,1990
...,...,...,...,...,...,...,...,...,...,...,...,...
52198,2014-12,YISHUN,5 ROOM,816,YISHUN ST 81,10 TO 12,122.0,Improved,1988,580000.0,NaN,2014
52199,2014-12,YISHUN,EXECUTIVE,325,YISHUN CTRL,10 TO 12,146.0,Maisonette,1988,540000.0,NaN,2014
52200,2014-12,YISHUN,EXECUTIVE,618,YISHUN RING RD,07 TO 09,164.0,Apartment,1992,738000.0,NaN,2014
52201,2014-12,YISHUN,EXECUTIVE,277,YISHUN ST 22,07 TO 09,152.0,Maisonette,1985,592000.0,NaN,2014


In [48]:
# what is the average price per year?
df.groupby('year', as_index=False).resale_price.mean()

,year,resale_price
0,1990,68086.142263
1,1991,73588.938934
2,1992,84372.635731
3,1993,132807.719750
4,1994,172619.529936
5,1995,212122.103228
6,1996,279282.337753
7,1997,308213.918543
8,1998,257637.912164
9,1999,252301.345534


Hm, but this does not account for flat type. Let's alter the analysis to include this.

In [52]:
tmp_df = df.groupby(['year','flat_type'],
           as_index=False).resale_price.agg(['count','mean','std'])

In [53]:
# need this to display all rows
with pd.option_context('display.max_rows', None): 
  display(tmp_df)

count           mean            std
year flat_type                                            
1990 1 ROOM               74    7890.540541    1222.412719
     2 ROOM              328   19571.097561    5889.986164
     3 ROOM             6992   42618.447941   11937.966834
     4 ROOM             3042   81765.902696   18265.041139
     5 ROOM             1636  132614.605746   28626.691026
     EXECUTIVE           433  186458.267898   26780.669542
1991 1 ROOM               51    8501.568627    1380.306303
     2 ROOM              233   21257.811159    6798.745505
     3 ROOM             6822   44479.893433   11499.105129
     4 ROOM             3405   81799.517474   20117.357584
     5 ROOM             1685  138102.878338   32421.508851
     EXECUTIVE           657  191166.849315   30184.763050
     MULTI GENERATION      2  165000.000000   21213.203436
1992 1 ROOM               49    9648.979592    2321.558604
     2 ROOM              288   22828.472222    7640.035830
     3 ROOM             7143   48173.884642   13963.040140
     4 ROOM             4144   90928.365347   25817.989947
     5 ROOM             2024  154383.896245   38429.557856
     EXECUTIVE           833  213970.979592   38699.504011
     MULTI GENERATION     22  226586.363636   52211.495682
1993 1 ROOM               52   17634.615385    7300.198349
     2 ROOM              353   35625.552408   11498.391923
     3 ROOM             7864   71018.616862   22651.569086
     4 ROOM             5430  134510.933517   37872.435685
     5 ROOM             2973  227253.050118   56251.022691
     EXECUTIVE          1394  296455.337877   54411.267763
     MULTI GENERATION     50  293700.000000   63951.272905
1994 1 ROOM               81   24824.691358    6018.794125
     2 ROOM              418   45405.502392    9042.971931
     3 ROOM            10968   94328.029085   20086.649204
     4 ROOM             8318  175071.872686   30138.963041
     5 ROOM             4579  284071.676130   45972.771239
     EXECUTIVE          1972  367723.477688   43561.971514
     MULTI GENERATION     37  398702.702703   65091.331077
1995 1 ROOM               79   37398.734177    7933.368245
     2 ROOM              439   59497.694761   10543.973350
     3 ROOM            10769  121297.863961   25087.146580
     4 ROOM             9598  216663.336216   39389.651920
     5 ROOM             4583  341553.173249   56487.158539
     EXECUTIVE          1794  443288.013378   59395.060127
     MULTI GENERATION     27  486585.185185   59196.801523
1996 1 ROOM              110   57417.272727   21764.864620
     2 ROOM              604   85941.847682   15407.707605
     3 ROOM            14549  173339.589250   36452.693359
     4 ROOM            12364  295949.042219   56760.750718
     5 ROOM             5069  441540.769185   68664.213228
     EXECUTIVE          2177  572772.335783   71151.083151
     MULTI GENERATION     46  606695.652174   83208.137841
1997 1 ROOM              112   72173.553571   20477.752355
     2 ROOM              511  108069.960861   15882.350014
     3 ROOM            10700  195114.824673   33472.765177
     4 ROOM            12450  304971.573414   47052.463936
     5 ROOM             5522  442181.538935   55575.350741
     EXECUTIVE          2427  567156.357643   54205.465668
     MULTI GENERATION     37  605918.918919   70209.360874
1998 1 ROOM               80   48706.250000    7940.292915
     2 ROOM              474   82706.782700   14476.648305
     3 ROOM            18143  154962.904867   26406.730444
     4 ROOM            18908  253756.264544   40699.407647
     5 ROOM             9071  373742.764304   48480.178439
     EXECUTIVE          4389  479759.779221   49218.771369
     MULTI GENERATION     30  516929.600000   61939.518848
1999 1 ROOM              139   50735.251799   12195.608994
     2 ROOM              492   77728.414634   14644.590290
     3 ROOM            19156  146116.696544   26875.066395
     4 ROOM            20862  243891.432796   40044.841178
     5 ROOM